# Migrationshintergrund

- Line/Barchart
- Gibt es andere Beeinträchtigungen?

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns
import plotly.express as px

## Import Data

In [9]:
file_path = "data.xlsx"

TypeError: _LocationIndexer.__call__() got multiple values for argument 'axis'

In [32]:
# Alle Sheets einlesen, aber das erste Blatt ignorieren
sheets = pd.read_excel(file_path, sheet_name=None, engine="openpyxl")

# Das erste Blatt entfernen
sheets.pop(next(iter(sheets)))  # Entfernt das erste Blatt basierend auf der Reihenfolge

# Dictionary zur Speicherung der Werte
gewichteter_anteil = {}  # Gewichtete relative Quote basierend auf der tatsächlichen Gesamtzahl

# Durch alle verbleibenden Jahre iterieren
for jahr, df in sheets.items():
    df.columns = df.columns.astype(str).str.strip()
    
    # Extrahiere die Zeilen mit "Migrationshintergrund (%)"
    migration_row = df[df["Variable"].str.contains("Migrationshintergrund", na=False) & 
                       df["Category"].str.contains("\(\%\)", na=False)]
    
    # Extrahiere die Zeilen mit "Anzahl Studierende"
    total_students_row = df[df["Variable"].str.contains("Rücklaufquote", na=False) & 
                            df["Category"].str.contains("Anzahl Studierende", na=False)]
    
    if not migration_row.empty and not total_students_row.empty:
        # Gesamtanzahl der Studierenden summieren
        total_students = total_students_row.iloc[:, 2:].sum(axis=1).values[0]

        # Mittelwert des Anteils über alle Studiengänge berechnen
        avg_migration_rate = migration_row.iloc[:, 2:].mean(axis=1).values[0]

        # Gewichteter relativer Anteil berechnet als: (absolute Anzahl / Gesamtzahl der Studierenden) * 100
        gewichteter_anteil[jahr] = (avg_migration_rate / 100) * 100

# Daten als DataFrame formatieren
df_migration = pd.DataFrame({
    "Jahr": list(gewichteter_anteil.keys()),
    "Gewichteter Relativer Anteil (%)": list(gewichteter_anteil.values())
})

df_migration = df_migration.sort_values(by="Jahr")

# Plot mit Plotly erstellen
fig = px.line(
    df_migration, 
    x="Jahr", 
    y="Gewichteter Relativer Anteil (%)", 
    markers=True, 
    title="Relativer Anteil Studierender mit Migrationshintergrund",
    color_discrete_sequence=px.colors.qualitative.Safe
)

fig.update_layout(
    xaxis_title="Jahr",
    yaxis_title="Relativer Anteil (%)",
    yaxis=dict(range=[0, 40]),
    template="plotly_white"
)

fig.show()

<>:22: SyntaxWarning:

invalid escape sequence '\('

<>:22: SyntaxWarning:

invalid escape sequence '\('

/var/folders/9v/2nh1cygd6qv3_k0knr9c4y_80000gn/T/ipykernel_27882/2808724614.py:22: SyntaxWarning:

invalid escape sequence '\('



In [158]:
import pandas as pd
import plotly.express as px

# Datei einlesen (ersetze 'data.xlsx' durch den tatsächlichen Dateinamen)
file_path = "data.xlsx"

# Alle Sheets einlesen, aber das erste Blatt ignorieren
sheets = pd.read_excel(file_path, sheet_name=None, engine="openpyxl")

# Das erste Blatt entfernen
sheets.pop(next(iter(sheets)))  # Entfernt das erste Blatt basierend auf der Reihenfolge

# Dictionary zur Speicherung der Werte
gewichteter_anteil = {}  # Gewichtete relative Quote basierend auf der tatsächlichen Gesamtzahl

# Durch alle verbleibenden Jahre iterieren
for jahr, df in sheets.items():
    df.columns = df.columns.astype(str).str.strip()
    
    # Extrahiere die Zeilen mit "Migrationshintergrund (%)"
    migration_row = df[df["Variable"].str.contains("Migrationshintergrund", na=False) & 
                       df["Category"].str.contains("\(\%\)", na=False)]
    
    # Extrahiere die Zeilen mit "Anzahl Studierende"
    total_students_row = df[df["Variable"].str.contains("Rücklaufquote", na=False) & 
                            df["Category"].str.contains("Anzahl Studierende", na=False)]
    
    if not migration_row.empty and not total_students_row.empty:
        # Gesamtanzahl der Studierenden summieren
        total_students = total_students_row.iloc[:, 2:].sum(axis=1).values[0]

        # Mittelwert des Anteils über alle Studiengänge berechnen
        avg_migration_rate = migration_row.iloc[:, 2:].mean(axis=1).values[0]

        # Gewichteter relativer Anteil berechnet als: (absolute Anzahl / Gesamtzahl der Studierenden) * 100
        gewichteter_anteil[jahr] = (avg_migration_rate / 100) * 100

# Daten als DataFrame formatieren
df_migration = pd.DataFrame({
    "Jahr": list(gewichteter_anteil.keys()),
    "Gewichteter Relativer Anteil (%)": list(gewichteter_anteil.values())
})

df_migration = df_migration.sort_values(by="Jahr")

# Plot mit Plotly erstellen
fig = px.line(
    df_migration, 
    x="Jahr", 
    y="Gewichteter Relativer Anteil (%)", 
    markers=True, 
    title="Relativer Anteil Studierender mit Migrationshintergrund",
    color_discrete_sequence=px.colors.qualitative.Safe,
    template="infoviz"
)

# Layout-Anpassungen
fig.update_traces(line=dict(width=3))  # Linie dicker machen

fig.update_layout(
    yaxis=dict(range=[0, 40]),  # Y-Achsenbereich festlegen
    annotations=[
        dict(
            text="Datenquelle: Universität 2024",  # Individueller Untertitel
            xanchor="left",  # Linksbündig
            x=-0.045,  # x-Position
            y=1.14,  # y-Position
            xref="paper", yref="paper",
            showarrow=False,
            font=dict(size=14, color="grey", family="Arial")  # Schriftgröße, Farbe und Schriftart
        )
    ],
    xaxis=dict(range=[2013, 2025]) 
)

fig.show()

<>:22: SyntaxWarning:

invalid escape sequence '\('

<>:22: SyntaxWarning:

invalid escape sequence '\('

/var/folders/9v/2nh1cygd6qv3_k0knr9c4y_80000gn/T/ipykernel_27882/519686091.py:22: SyntaxWarning:

invalid escape sequence '\('



In [152]:
import plotly.io as pio

# Benutzerdefiniertes Template definieren
infoviz_template = dict(
    layout=dict(
        template="plotly_white",
        title=dict(
            font=dict(size=20, family="Arial", weight="bold", color="black"),
            y=0.91,  # Titel weiter nach oben
            x=0.05,  # Links ausgerichtet
            xanchor="left",  # Linksbündig
        ),
        xaxis=dict(
            showgrid=False,
            zerolinecolor="lightgrey",
            tickfont=dict(color="grey", size=12),
            title_font=dict(color="grey", weight="bold", size=13),
            title_standoff=15,
            ticklabelposition="outside bottom"
        ),
        yaxis=dict(
            showgrid=True, gridcolor="lightgrey",
            zerolinecolor="lightgrey",
            tickfont=dict(color="grey", size=12),
            title_font=dict(color="grey", weight="bold", size=13),
            title_standoff=15,
            ticklabelposition="outside left"
        ),
    )
)
pio.templates["infoviz"] = infoviz_template